In [ ]:
from aiida import load_profile, common, orm 
load_profile();

from aiida.orm import QueryBuilder
import ipywidgets as widgets
from IPython.display import display
from widgets import query, misc
import aiidalab_widgets_base as awb
from settings import *
import pandas as pd
from utils import make_query

from IPython.core.display import display, HTML
display(HTML("""<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:10000px;
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>"""))

# Inversion results

Remote directories of successful inversions. 

In [ ]:
extra_labels = widgets.Dropdown(
            options = make_query.get_extra_(INVERSION_WORKLFOW, name=None)+['TEST']
        )
extra_labels

In [ ]:
def get_data_inversions():
    qb = QueryBuilder()
    qb.append(
                INVERSION_WORKLFOW,
                tag = 'inv_w',
                filters={"attributes.exit_status": 0,
                         "extras": {"has_key": extra_labels.value}},
                project=["id"],
            ).append(
                INVERSION,
                tag = 'inv',
                with_incoming = 'inv_w', 
            )
    qb.append(orm.Dict,
            with_outgoing = 'inv',
            project = ['attributes.run_str','attributes.dtm_start','attributes.dtm_end'])
    qb.append(orm.RemoteData, 
            with_incoming = 'inv',
            project = ['attributes.remote_path'])
    return qb.all()

In [ ]:

def gen_table():
    html = """<style>
            table, th, td {
                border: 1px solid;
                text-align: center;
                padding: 5px}</style>
            <table>
                    <tr>
                        <th>Id</th>
                        <th>run.str</th>
                        <th>dtm.start</th>
                        <th>dtm.end</th>
                        <th>Remote address</th>
                    </tr>"""
    for i in get_data_inversions(): 
        html+=f"""  <td>{i[0]}</td>
                    <td>{i[1]}</td>
                    <td>{i[2]}</td>
                    <td>{i[3]}</td>
                    <td>{i[4]}</td>
                    </tr>"""
    html += "</table>"
    return html

In [ ]:
search_b = widgets.Button(description = 'Search',
                          button_style='info')
def on_button_clicked(b):
    table = widgets.HTML(gen_table())
    display(table)
    
search_b.on_click(on_button_clicked)
search_b